## Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report


## Data Collection and Pre-Processing

In [3]:
df = pd.read_csv('Crime_Data_from_2020_to_Present (2).csv')
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'Crime_Data_from_2020_to_Present (2).csv'

In [2]:
df.columns

NameError: name 'df' is not defined

In [3]:
df[['Mocodes',
 'Vict Sex',
 'Vict Descent',
 'Premis Cd',
 'Premis Desc',
 'Weapon Used Cd',
 'Weapon Desc',
 'Crm Cd 1',
 'Crm Cd 2',
 'Crm Cd 3',
 'Crm Cd 4',
 'Cross Street']]

,Mocodes,Vict Sex,Vict Descent,Premis Cd,Premis Desc,Weapon Used Cd,Weapon Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,Cross Street
0,NaN,M,O,101.0,STREET,NaN,NaN,510.0,998.0,NaN,NaN,NaN
1,1822 1402 0344,M,O,128.0,BUS STOP/LAYOVER (ALSO QUERY 124),NaN,NaN,330.0,998.0,NaN,NaN,NaN
2,0344 1251,X,X,502.0,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)",NaN,NaN,480.0,NaN,NaN,NaN,NaN
3,0325 1501,M,O,405.0,CLOTHING STORE,NaN,NaN,343.0,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,101.0,STREET,NaN,NaN,510.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
986868,NaN,NaN,NaN,101.0,STREET,NaN,NaN,510.0,NaN,NaN,NaN,NaN
986869,0329 0400 0416,X,X,503.0,HOTEL,500.0,UNKNOWN WEAPON/OTHER WEAPON,745.0,NaN,NaN,NaN,NaN
986870,0344,M,B,210.0,RESTAURANT/FAST FOOD,NaN,NaN,341.0,NaN,NaN,NaN,NaN
986871,1822 0334 0416 0445 0449 1202,F,W,102.0,SIDEWALK,308.0,STICK,230.0,NaN,NaN,NaN,JEFFERSON BL


In [ ]:
reduced_crime_df = df
# Saving the selected DataFrame to a CSV file
reduced_crime_df.to_csv("reduced_crime_df.csv", index=False)

print("CSV file 'reduced_crime_df.csv' saved successfully!")


In [4]:
df['Vict Sex'].fillna('X', inplace = True)

In [5]:
df['Vict Sex'].replace(['H', '-'], 'X', inplace = True)

In [7]:
df['Vict Descent'].replace('-', 'Unknown', inplace = True)

In [6]:
df[['Vict Descent', 'Weapon Used Cd', 'Weapon Desc']] = df[['Vict Descent', 'Weapon Used Cd', 'Weapon Desc']].fillna('Unknown')

In [7]:
df.dropna(subset = ['Crm Cd 1'], inplace = True)

In [8]:
# Add case_solved column
df['case_solved'] = df['Status Desc'].apply(lambda x: 'Not solved' if x == 'Invest Cont' else 'Solved')

# Data preparation for victim_sex plot
sex_solved_counts = df.groupby(['Vict Sex', 'case_solved']).size().unstack()



In [9]:
# assining variable as DataFrame
area_crime_counts = df['AREA NAME'].value_counts().to_frame()

area_under_25 = area_crime_counts[(area_crime_counts >= 29796) & (area_crime_counts < 37085)].dropna().index.to_list()
area_under_50 = area_crime_counts[(area_crime_counts >= 37085) & (area_crime_counts < 42548)].dropna().index.to_list()
area_under_75 = area_crime_counts[(area_crime_counts >= 42548) & (area_crime_counts < 45503)].dropna().index.to_list()
area_over_75 = area_crime_counts[area_crime_counts >= 45503].dropna().index.to_list()

print(f'Areas of min-25 percentage: {area_under_25}')
print(f'Areas of 25-50 percentage: {area_under_50}')
print(f'Areas of 50-75 percentage: {area_under_75}')
print(f'Areas of 75-max percentage: {area_over_75}')

print("----------------------------------------------------\n")

print(area_crime_counts.loc[area_under_25])
print(area_crime_counts.loc[area_under_50])
print(area_crime_counts.loc[area_under_75])
print(area_crime_counts.loc[area_over_75])


Areas of min-25 percentage: ['Hollenbeck']
Areas of 25-50 percentage: ['West Valley', 'Harbor', 'Devonshire', 'Topanga', 'Mission']
Areas of 50-75 percentage: ['West LA', 'Van Nuys', 'Northeast']
Areas of 75-max percentage: ['Central', '77th Street', 'Pacific', 'Southwest', 'Hollywood', 'Southeast', 'Newton', 'Olympic', 'Wilshire', 'Rampart', 'N Hollywood']
----------------------------------------------------

            count
AREA NAME        
Hollenbeck  36755
             count
AREA NAME         
West Valley  41878
Harbor       41070
Devonshire   40874
Topanga      40546
Mission      40317
           count
AREA NAME       
West LA    45272
Van Nuys   42642
Northeast  42576
             count
AREA NAME         
Central      68959
77th Street  61646
Pacific      58881
Southwest    56972
Hollywood    52107
Southeast    50101
Newton       48917
Olympic      48519
Wilshire     47942
Rampart      46520
N Hollywood  46174


# Splicting data for features and target

In [10]:
low_risk = area_crime_counts[area_crime_counts <= 42548].dropna().index.to_list()
high_risk = area_crime_counts[area_crime_counts > 42548].dropna().index.to_list()

# Add risk column
df['Risk'] = df['AREA NAME'].apply(lambda area: 'Low risk' if area in low_risk else 'High risk')

In [12]:
################NEW#########################
# Add violent crime  column

violent_codes = [231,230,220,624,622,623,860,235,627,814,922,110,822,921,830,236,626,910,920,435,113,625,122,121,210,810,815,251,250,821]
for value in violent_codes:
    df['Violent_Crime'] = (df['Crm Cd'] == value).astype(int)

In [14]:
################NEW#########################
features = df[['AREA', 'Crm Cd', 'Vict Sex', 'Vict Descent', 'Violent_Crime']]

In [16]:
################NEW#########################
df_cat = features.select_dtypes(include = 'object')
df_num = features.select_dtypes(include = [np.number])

In [20]:
################NEW#########################
from sklearn.preprocessing import LabelEncoder
Encoder = LabelEncoder()

for col in df_cat:
    if col not in ['Violent_Crime']:
        features[col] = Encoder.fit_transform(features[col])



In [22]:
################NEW#########################
features.corr()

,AREA,Crm Cd,Vict Sex,Vict Descent,Violent_Crime
AREA,1.000000,-0.004759,-0.027656,0.004416,-0.002373
Crm Cd,-0.004759,1.000000,-0.043703,-0.010354,0.036064
Vict Sex,-0.027656,-0.043703,1.000000,0.487856,-0.011671
Vict Descent,0.004416,-0.010354,0.487856,1.000000,-0.009715
Violent_Crime,-0.002373,0.036064,-0.011671,-0.009715,1.000000


## Taking Sample data for Predection

In [23]:
################NEW#########################
# Sample 300,000 rows from the DataFrame
sampled_df = features.sample(n=300000, random_state=42)

In [25]:
################NEW#########################
X = sampled_df[['AREA', 'Crm Cd', 'Vict Sex', 'Vict Descent']]
y = sampled_df['Violent_Crime']

## Scaling the Dataset

In [26]:
################NEW#########################
# import StandarScaler from sklearn
from sklearn.preprocessing import StandardScaler

# initialize the standard scalar
Scale = StandardScaler()

# scale all the numeric variables
# standardize all the columns of the dataframe 'X_Scaled'
X_Scaled = Scale.fit_transform(X)

# create a dataframe of scaled numerical variables
# pass the required column names to the parameter 'columns'
X = pd.DataFrame(X_Scaled, columns = X.columns)

## Splting Data into Train and Test

In [27]:
################NEW#########################
# split data into train subset and test subset
# set 'random_state' to generate the same dataset each time you run the code
# 'test_size' returns the proportion of data to be included in the testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 21, test_size = 0.2)

# check the dimensions of the train & test subset using 'shape'
# print dimension of train set
print('X_train', X_train.shape)
print('y_train', y_train.shape)

# print dimension of test set
print('X_test', X_test.shape)
print('y_test', y_test.shape)

X_train (240000, 4)
y_train (240000,)
X_test (60000, 4)
y_test (60000,)


## Logistic Regression

In [28]:
################NEW#########################
model = LogisticRegression()
model.fit(X_train,y_train)

# train accuracy
X_train_prediction  = model.predict(X_train)
train_data_accuracy = accuracy_score(X_train_prediction,y_train)

print('Accuracy score of our train data:',train_data_accuracy)

# test accuracy
X_test_prediction  = model.predict(X_test)
train_data_accuracy = accuracy_score(X_train_prediction, y_train)
test_data_accuracy = accuracy_score(X_test_prediction,y_test)

print('Accuracy score of our test data:',test_data_accuracy)

Accuracy score of our train data: 0.9993958333333334
Accuracy score of our test data: 0.9994833333333333


In [29]:
################NEW#########################
# You can also print out classification reports for more detailed performance analysis
print("Logistic Regression Classification Report:\n", classification_report(y_train, X_train_prediction))

Logistic Regression Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00    239855
           1       0.00      0.00      0.00       145

    accuracy                           1.00    240000
   macro avg       0.50      0.50      0.50    240000
weighted avg       1.00      1.00      1.00    240000



In [30]:
################NEW#########################
# You can also print out classification reports for more detailed performance analysis
print("Logistic Regression Classification Report:\n", classification_report(y_test, X_test_prediction))

Logistic Regression Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     59969
           1       0.00      0.00      0.00        31

    accuracy                           1.00     60000
   macro avg       0.50      0.50      0.50     60000
weighted avg       1.00      1.00      1.00     60000



ANALYSIS of the ABOVE
What Does This Mean?
Training Accuracy: 0.9994 (99.94%):

The model performs extremely well on the training dataset, correctly predicting almost all cases.
Testing Accuracy: 0.9995 (99.95%):

The model also performs very well on unseen data, indicating strong generalization.


**************Imbalanced Dataset: If your dataset is imbalanced (e.g., one class dominates), such high accuracy might be misleading. 


Key Insights
Class Imbalance:

Class 0 (majority): 239,855 instances
Class 1 (minority): 145 instances
The dataset is highly imbalanced, with Class 0 dominating.
Precision, Recall, F1-Score:

Class 0:
Precision, recall, and F1-score are all 1.00, meaning the model correctly identifies almost all Class 0 instances.
Class 1:
Precision, recall, and F1-score are all 0.00, indicating the model completely fails to identify Class 1 instances.
Accuracy:

The overall accuracy is 1.00 (99.94%), but this is misleading because it only reflects the correct predictions for the majority class (Class 0).
Macro Average:

The macro average for precision, recall, and F1-score is 0.50, which highlights the disparity in performance between the two classes.
Weighted Average:

Weighted metrics are dominated by the majority class, giving an illusion of good performance.

How to Address This
Resampling Techniques:

Oversample the Minority Class: Add synthetic or duplicate samples for Class 1 using techniques like SMOTE:

In [ ]:
################NEW#########################
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Check the class distribution before and after SMOTE
print("Before SMOTE:")
print(y_train.value_counts())
print("After SMOTE:")
print(pd.Series(y_train_resampled).value_counts())


Before SMOTE:
Violent_Crime
0    239855
1       145
Name: count, dtype: int64
After SMOTE:
Violent_Crime
0    239855
1    239855
Name: count, dtype: int64


In [ ]:
################NEW#########################
model = LogisticRegression()
model.fit(X_train_resampled,y_train_resampled)

# train accuracy
X_train_prediction  = model.predict(X_train_resampled)
train_data_accuracy = accuracy_score(X_train_prediction,y_train_resampled)

print('Accuracy score of our train data:',train_data_accuracy)

# test accuracy
X_test_prediction  = model.predict(X_test)
train_data_accuracy = accuracy_score(X_train_prediction, y_train_resampled)
test_data_accuracy = accuracy_score(X_test_prediction,y_test)

print('Accuracy score of our test data:',test_data_accuracy)

Accuracy score of our train data: 0.9242563215275896
Accuracy score of our test data: 0.8498


In [ ]:
################NEW#########################
# You can also print out classification reports for more detailed performance analysis
print("Logistic Regression Classification Report:\n", classification_report(y_train_resampled, X_train_prediction))

Logistic Regression Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.85      0.92    239855
           1       0.87      1.00      0.93    239855

    accuracy                           0.92    479710
   macro avg       0.93      0.92      0.92    479710
weighted avg       0.93      0.92      0.92    479710



analysis from above
The Logistic Regression Classification Report you provided indicates a significant improvement in the performance of your model after applying SMOTE for oversampling the minority class. Here's a breakdown of the results:
Key Metrics Breakdown
Class 0 (Majority Class)

Precision: 1.00
Recall: 0.85
F1-Score: 0.92
Support: 239,855 instances
This means the model is very precise when predicting Class 0 (i.e., correctly classifies instances of Class 0 most of the time), but it misses 15% of Class 0 cases (lower recall).

Class 1 (Minority Class)

Precision: 0.87
Recall: 1.00
F1-Score: 0.93
Support: 239,855 instances
For Class 1 (the minority class), the model now correctly classifies all instances (perfect recall), but there are still some errors in the predictions, reflected by the slightly lower precision (0.87).

Accuracy:

Accuracy: 0.92 (92%)
Overall, the model performs well on both classes, but there is a trade-off between precision and recall, especially for the majority class.
Macro Average:

Precision: 0.93
Recall: 0.92
F1-Score: 0.92
The macro average calculates the average across all classes without taking class imbalance into account. It gives you a general idea of the model's overall performance across both classes.

Weighted Average:

Precision: 0.93
Recall: 0.92
F1-Score: 0.92
The weighted average accounts for the imbalanced class distribution, giving more weight to the majority class. This is a more reflective measure of performance when the classes are imbalanced.

Key Takeaways:
Class 1 (minority class) now has perfect recall (1.00), which means it is fully captured by the model, but there’s a slight drop in precision (0.87). This indicates some false positives (Class 0 instances incorrectly predicted as Class 1).
Class 0 (majority class) still has a very high precision (1.00), but its recall has dropped slightly to 0.85. This suggests the model might be missing some Class 0 instances (false negatives).
Overall Accuracy: 92% is a solid performance, but the F1-scores of both classes indicate that there might still be room for improvement, especially in achieving a better balance between precision and recall.